From a design perspective, deep hierarchies of classes can be cumbersome and make change a lot harder since the entire hierarchy has to be taken into account. Python offers a few mechanism to avoid this, and make the class desing leaner.

In order to ensure that all cells in this notebook can be evaluated without errors, we will use `try`-`except` to catch exceptions. To show the actual errors, we need to print the backtrace, hence the following import.

In [1]:
from traceback import print_exc

# Duck typing

The idea is that if classes implement object methods with the same signature and semantics, that functionality can be used, regardless of the relationship between the classes, if any. If it looks like a duck, swims like a duck and quacks like a duck, it is probably a duck.

We define two classes that serve completely different purposes.  The only thing they share is that they make a sound, and the relevant method for both is `make_sound`.

In [2]:
class Duck:
    species: str
    
    def make_sound(self):
        return 'quack'
    
    def __init__(self, species):
        self.species = species

In [3]:
class Timer:
    time: int
    
    def make_sound(self):
        return 'ring'
    
    def __init__(self, time):
        self.time = time

Next, we add and instance of each to a list.

In [4]:
stuff = [Duck('mandarin'), Timer(10)]

We can iterate over the list, and regardless of the object's class, invoke the `make_sound` method.

In [5]:
for item in stuff:
    print(f'{type(item)} says {item.make_sound()}')

<class '__main__.Duck'> says quack
<class '__main__.Timer'> says ring


Note that the sound faculty of these classes is not derived from a common ancestor class by inheritance.

# Protocols

Duck typing can be formalized in Python 3.8+ using protocols.  This will allow type checkers such as `mypy` to find potential mistakes.  Consider the code as above, but now implemented using a `Protocol`.

A protocol is defined as a class that inherits from `Protocol` that is defined in the `typing` module.  It defines the method signatures that should be implemented by any class that supports the protocol.  In the example below, any class that can make a sound should have a `make_sound` method that returns a `str`.

The classes that implement the protocol `SoundMaker` *do not* inherit from that class, they simply implement the `make_sound` method as specified by the protocol, i.e., a method that takes no arguments besides the object, and that returns a `str`.

The `make_sound` function takes an object of type `SoundMaker` as an argument.  Although neither `Duck` nor `Timer` inherit from `SoundMaker`, the type checker will nevertheless be satisfied since both classes implement the protocol `SoundMaker` since they have a `make_sound` method implementation.

Since the `Dog` class doesn't implement `make_sound`, a static type checker will now report an error when a `Dog` object is passed as an argument to the `make_sound` function.

In [6]:
%%writefile protocols_toremove.py
#!/usr/bin/env python

from typing import Protocol


class SoundMaker(Protocol):
    def make_sound(self) -> str: ...


class Duck:
    species: str

    def __init__(self, species: str):
        self.species = species

    def make_sound(self) -> str:
        return 'quack'


class Timer:
    time: int

    def __init__(self, time: int):
        self.time = time

    def make_sound(self) -> str:
        return 'ring'


class Dog:
    name: str

    def __init__(self, name: str):
        self.name = name


def make_sound(stuff: SoundMaker) -> None:
    print(stuff.make_sound())


if __name__ == "__main__":
    duck = Duck('Mallard')
    timer = Timer(5)
    dog = Dog('Fido')

    make_sound(duck)
    make_sound(timer)

    things: list[SoundMaker] = [duck, timer]
    for thing in things:
        make_sound(thing)

    # Does't pass type check, will result in runtime error 
    make_sound(dog)

Writing protocols_toremove.py


Now we can run `mypy` to do type checking.

In [7]:
!mypy protocols_toremove.py

protocols_toremove.py:54: error: Argument 1 to "make_sound" has incompatible type "Dog"; expected "SoundMaker"  [arg-type]
Found 1 error in 1 file (checked 1 source file)


Indeed, running this script would result in a runtime error.

In [8]:
!python protocols_toremove.py

quack
ring
quack
ring
Traceback (most recent call last):
  File "/home/gjb/Projects/Python-software-engineering/source-code/object-orientation/protocols_toremove.py", line 54, in <module>
    make_sound(dog)
  File "/home/gjb/Projects/Python-software-engineering/source-code/object-orientation/protocols_toremove.py", line 38, in make_sound
    print(stuff.make_sound())
          ^^^^^^^^^^^^^^^^
AttributeError: 'Dog' object has no attribute 'make_sound'


In [9]:
!rm protocols_toremove.py

# Mix-in

If the implementation of the common functionality is the same for a number of classes, it is worth defining a mix-in class that defines the implementation.  In the examples above, the `make_sound` method implementation is identical for the `Duck` and `Computer` class. Hence we can move the implementation to its own class `SoundMake`. Note that this class has no `__init__` method, and needs none.

In [10]:
class SoundMaker:
    
    def make_sound(self):
        if hasattr(self, '_sound'):
            return self._sound
        else:
            raise ValueError(f'{type(self)} does not make sound')

The `Duck`, `Timer` and `Dog` classes now inherit from `SoundMaker`, but `Dog` doesn't define its sound attribute.

In [11]:
class Duck(SoundMaker):
       
    _sound: str = 'quack'
    species: str
    
    def __init__(self, species):
        self.species = species

In [12]:
class Timer(SoundMaker):
    
    _sound: str = 'beep'
    time: int
    
    def __init__(self, time):
        self.time = time

In [13]:
class Dog(SoundMaker):
    
    name: str
        
    def __init__(self, name):
        self.name = name

In [14]:
stuff = [Duck('mandarin'), Timer(5)]

In [15]:
for item in stuff:
    print(f'{type(item)} says {item.make_sound()}')

<class '__main__.Duck'> says quack
<class '__main__.Timer'> says beep


Since the `Dog` has no `_sound`, the mix-in method raises an exception.

In [16]:
dog = Dog('fido')
try:
    print(dog.make_sound())
except ValueError as error:
    print_exc()

Traceback (most recent call last):
  File "/tmp/ipykernel_3319/3334719555.py", line 3, in <module>
    print(dog.make_sound())
          ^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_3319/2135747159.py", line 7, in make_sound
    raise ValueError(f'{type(self)} does not make sound')
ValueError: <class '__main__.Dog'> does not make sound


However, note that this implies a class hierarchy.